### 학습목표
- 감성분석의 원리를 이해하고 활용할 수 있다.
- 임베딩방법론을 이해하고 활용할 수 있다.

#### 텍스트마이닝
 - 비정형 텍스트 데이터에서 의미를 추출하는 작업

#### 1. 감성분석
- 텍스트데이터에서 작성자의 기분, 태도, 감정 등을 파악하는 분석
- 감성분석을 하는 방법은 크게 2가지가 있음(감성사전을 사전에 구축하고 단어의 빈도를 측정하는 방법, 인공지능 모델을 이용해 긍/부정 문장을 학습시키는 방법)

https://github.com/smilegate-ai/korean_unsmile_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
%cd "/content/drive/MyDrive/LGDXSCHOOL 2기/신규가치분석"

/content/drive/MyDrive/LGDXSCHOOL 2기/신규가치분석


In [13]:
import pandas as pd

train = pd.read_csv('./data/unsmile_train_v1.0.tsv', delimiter='\t')
test = pd.read_csv('./data/unsmile_valid_v1.0.tsv', delimiter='\t')

In [14]:
display(train.info())
display(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15005 entries, 0 to 15004
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      15005 non-null  object
 1   여성/가족   15005 non-null  int64 
 2   남성      15005 non-null  int64 
 3   성소수자    15005 non-null  int64 
 4   인종/국적   15005 non-null  int64 
 5   연령      15005 non-null  int64 
 6   지역      15005 non-null  int64 
 7   종교      15005 non-null  int64 
 8   기타 혐오   15005 non-null  int64 
 9   악플/욕설   15005 non-null  int64 
 10  clean   15005 non-null  int64 
 11  개인지칭    15005 non-null  int64 
dtypes: int64(11), object(1)
memory usage: 1.4+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3737 entries, 0 to 3736
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   문장      3737 non-null   object
 1   여성/가족   3737 non-null   int64 
 2   남성      3737 non-null   int64 
 3   성소수자    3737 non-null   int64 
 4   인종/국적   3737 non-null   int64 
 5   연령      3737 non-null   int64 
 6   지역      3737 non-null   int64 
 7   종교      3737 non-null   int64 
 8   기타 혐오   3737 non-null   int64 
 9   악플/욕설   3737 non-null   int64 
 10  clean   3737 non-null   int64 
 11  개인지칭    3737 non-null   int64 
dtypes: int64(11), object(1)
memory usage: 350.5+ KB


None

In [15]:
display(train)

,문장,여성/가족,남성,성소수자,인종/국적,연령,지역,종교,기타 혐오,악플/욕설,clean,개인지칭
0,일안하는 시간은 쉬고싶어서 그런게 아닐까,0,0,0,0,0,0,0,0,0,1,0
1,아동성범죄와 페도버는 기록바 끊어져 영원히 고통 받는다. 무슬림 50퍼 근친이다. ...,0,0,0,0,0,0,1,0,0,0,0
2,루나 솔로앨범 나왔을 때부터 머모 기운 있었음 ㅇㅇ Keep o doin 진짜 띵...,0,0,0,0,0,0,0,0,0,1,0
3,홍팍에도 어버이연합인가 보내요 뭐 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽...,0,0,0,0,0,0,0,0,0,1,0
4,아놔 왜 여기 댓들은 다 여자들이 김치녀라고 먼저 불렸다! 여자들은 더 심하게 그런...,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15000,께롱께롱께롱!!!,0,0,0,0,0,0,0,0,0,1,0
15001,갱상도가아니라 홍어지 개좃같은 홍어년들,0,0,0,0,0,1,0,0,0,0,0
15002,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,0,0,1,0,0,0,0,0,0,0
15003,링크도 안박고 가서 글 쓰자고 선동하네,0,0,0,0,0,0,0,0,1,0,0


#### 2. 데이터 클렌징
- 오타 및 띄어쓰기 교정
- 이모지 제거
- 불용어 등록 및 처리
- 정규표현식을 이용한 전처리 등

In [16]:
from tqdm import tqdm
import re # 정규표현식

In [20]:
# 패턴생성
p=re.compile("[ㅋㅎㄷㅇㅜㅠㅉ>~!?.0-9a-zA-Z]+")
train_clean=[] # 전처리된 전체 텍스트를 담을 리스트
for doc in tqdm(train['문장']): # 약 1만 5천번 반복
  temp=[] # 전처리된 문장의 토큰이 들어갈 리스트
  for token in doc.split(" "): # 한 문장을 띄어쓰기 중심으로 토큰화
    if p.search(token): # 정규표현식 패턴에 매칭되는지 확인
      continue
    if len(token)<2 : # 1글자 이상만
      continue
    temp.append(token)
  # 클렌징된 토큰 묶음을 하나의 문장으로 만들어서 추가
  train_clean.append(" ".join(temp))

100%|██████████| 15005/15005 [00:00<00:00, 156464.58it/s]


In [22]:
train_clean[:20]

['일안하는 시간은 쉬고싶어서 그런게 아닐까',
 '아동성범죄와 페도버는 기록바 끊어져 영원히 고통 무슬림 떨어지고 출산 위험은',
 '루나 솔로앨범 나왔을 때부터 머모 기운 있었음 진짜 띵곡임 들어보셈"',
 '홍팍에도 어버이연합인가 보내요 이런뎃글 있는데 이거 어버이연합측에 신고하면 그쪽에서 고소',
 '아놔 여기 댓들은 여자들이 김치녀라고 먼저 여자들은 심하게 이렇게 내가 둘다 나쁜 이상도 이하도 아닌데',
 '고향가서 피방가면 동네 부럴 친구들이랑은 거르는 없이 이야기하니까 말하게 되더라 당연히 키보드를 치거나 그러지는 않는데 말하는게 많이 거칠어지긴 반성해야겠네',
 '예수 십새끼 개새끼 창녀아들 애비실종 가정교육 못받은 무뇌충 허언증 정신병자 사기꾼 망상 근본없는 병신 장애인 짱개 흑인 이슬람 전라도랑 동급',
 '나이쳐먹고 피시방가는 놈들은 대가리에 똥만찬 놈들임',
 '출산청같은 우리나라 계집들 의무적으로 군대보내면 해결될 정신개조부터 지할꺼 다하고 무슨 애를 낳고 무슨 자식교육을',
 '재능 ㅂㅅㅌㅊ노 합성배우고싶다이기',
 '가짜 남자는 어떻게',
 '이슬람뺨치내',
 '어린이집에 월요일에 데려다주고 금요일에 데리고 가슴이 너무 아가가 얼마나',
 '갓슬람 정희구현 테러 ㅅㅅㅅㅅ미제양키새끼들에게는 테러가 답이다좆같은 양키 씨발새끼들',
 '따고 넘어갔나보네',
 '리더십, 메시지로 내부 결속을 강화하고 외부로 이미지를 구축하여 다시 교회로 모일수 있기를 소망하고 기도합니다',
 '협박스킬은 패시브랑께',
 '위로와 격려를 호텔에서 저여자도 정상은 아니구만 완전 지네 할아버지보다 나이가 많을듯한데 둘다 그렇게 더러운짓은 하고 설교를 어떻게',
 '메틸 알콜 집에서 쓰고 질문 에틸에 메틸 공업용 에틸 알콜은 면세라 불법적으로 술제조 때문에 정도 메틸이 들어가면서 제조방지용으로',
 '띄울사람이 진짜 없긴 없나보다']

####3.  토큰화 및 수치화
- 토큰화 : 일정 단위로 텍스트를 분리하는 작업
  - 단어(띄어쓰기)
  - 글자(char)
  - n-gram(유니,바이,트라이 등)
  - 형태소, 품사
- 수치화 : 의미있는 정보를 담고 있는 숫자형태로 변환하는 방법 -> 데이터를 정형화하는 효과가 있다
  - 빈도기반(랭킹, BOW, tf-idf)
  - embedding (word, document)

###### BOW (Bag Of Word)
- 문장에서 특정 단어의 등장 빈도를 수치화하는 방법
- 특징 1 : 문장에서 단어의 순서를 고려하기 않는다(문맥x -> 자연어처리에 부적합)
- 특징 2 : 말뭉치에서 사용되는 단어가 많으면 부피가 비례해서 커진다(의미있는 정보를 가지는 숫자는 적고 0과 같이 불필요한 정보량이 많아진다)

In [ ]:
# 1. 오늘 집에서 밥을 먹었다
# 2. 오늘 나는 집에 간다
# 3. 오늘 나는 밥을 먹었다
# 4. 내일 나는 밥을 먹었다
# 단어사전 총 8개의 단어로 이루어짐

In [23]:
from sklearn.feature_extraction.text import CountVectorizer # BOW를 구현할 클래스

In [24]:
sample_text = ["나는 오늘 저녁에 치킨을 먹을꺼야","너는 오늘 저녁에 무엇을 먹을꺼니?",
               "나는 오늘 아침에 운동을 하고 왔어","어제 저녁에 운동을 했더니 배가 많이 고프다"]

In [25]:
# step1 : 토큰화 및 단어사전 구축
sample_cv = CountVectorizer()
sample_cv.fit(sample_text)

CountVectorizer()

In [26]:
sample_cv.vocabulary_

{'나는': 1,
 '오늘': 10,
 '저녁에': 13,
 '치킨을': 14,
 '먹을꺼야': 5,
 '너는': 2,
 '무엇을': 6,
 '먹을꺼니': 4,
 '아침에': 8,
 '운동을': 12,
 '하고': 15,
 '왔어': 11,
 '어제': 9,
 '했더니': 16,
 '배가': 7,
 '많이': 3,
 '고프다': 0}